<a href="https://colab.research.google.com/github/Shejal987/chatbot/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
import nltk
import json
import io
import pandas as pd
import pickle
import random

In [ ]:
from google.colab import files
uploaded= files.upload()

Saving intents.json to intents.json


In [ ]:
uploaded

{'intents.json': b'{"intents": [\r\n        {"tag": "greeting",\r\n         "patterns": ["Hi", "How are you", "Is anyone there?", "Hello", "Good day", "Whats up"],\r\n         "responses": ["Hello!", "Good to see you again!", "Hi there, how can I help?"],\r\n         "context_set": ""\r\n        },\r\n        {"tag": "goodbye",\r\n         "patterns": ["cya", "See you later", "Goodbye", "I am Leaving", "Have a Good day"],\r\n         "responses": ["Sad to see you go :(", "Talk to you later", "Goodbye!"],\r\n         "context_set": ""\r\n        },\r\n        {"tag": "age",\r\n         "patterns": ["how old", "how old is tim", "what is your age", "how old are you", "age?"],\r\n         "responses": ["I am 18 years old!", "18 years young!"],\r\n         "context_set": ""\r\n        },\r\n        {"tag": "name",\r\n         "patterns": ["what is your name", "what should I call you", "whats your name?"],\r\n         "responses": ["You can call me Tim.", "I\'m Tim!", "I\'m Tim aka Tech With

In [ ]:
data = pd.read_json(io.StringIO(uploaded.get('intents.json').decode('utf-8')))

In [ ]:
data

,intents
0,"{'tag': 'greeting', 'patterns': ['Hi', 'How ar..."
1,"{'tag': 'goodbye', 'patterns': ['cya', 'See yo..."
2,"{'tag': 'age', 'patterns': ['how old', 'how ol..."
3,"{'tag': 'name', 'patterns': ['what is your nam..."
4,"{'tag': 'shop', 'patterns': ['Id like to buy s..."
5,"{'tag': 'hours', 'patterns': ['when are you gu..."


In [ ]:
from nltk.stem.lancaster import LancasterStemmer
stemmer= LancasterStemmer()

In [ ]:
nltk.download('punkt')

try:

  with open('data.pickle','rb') as f:
    words,labels,training,output= pickle.load()

except:
  words=[]
  doc_patterns=[]
  doc_tag=[]
  labels=[]

  for intent in data['intents']:
    for pattern in intent['patterns']:
      wrd= nltk.word_tokenize(pattern)
      words.extend(wrd)
      doc_patterns.append(wrd)
      doc_tag.append(intent['tag'])
    
    if intent['tag']  not in labels:
      labels.append(intent['tag'])  

  words= [stemmer.stem(w.lower()) for w in words if w != '?']
  words= sorted(list(set(words)))
  labels= sorted(labels)

  training=[]
  output=[]
  out_empty=[0 for _ in range(len(labels))]

  for x,doc in enumerate(doc_patterns):
    bag=[]

    wrds=[stemmer.stem(w.lower()) for w in doc]

    for w in words:
      if w in wrds:
        bag.append(1)
      else:
        bag.append(0)
  
    o_row= list(out_empty)
    o_row[labels.index(doc_tag[x])]=1

    training.append(bag)
    output.append(o_row)

  training= np.array(training)
  output= np.array(output)

  with open('data.pickle','wb') as f:
    pickle.dump((words,labels,training,output),f)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(8, input_shape=(len(training[0]),), activation='relu'))
model.add(tf.keras.layers.Dense(8, activation='relu'))
model.add(tf.keras.layers.Dense(len(output[0]), activation='softmax'))

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.SGD(0.01),
    loss='categorical_crossentropy',
    metrics=['accuracy'])

In [ ]:

model.fit(training,output,epochs=500,batch_size=8)

Epoch 1/500
4/4 [==============================] - 1s 4ms/step - loss: 1.8129 - accuracy: 0.1538
Epoch 2/500
4/4 [==============================] - 0s 3ms/step - loss: 1.8095 - accuracy: 0.1538
Epoch 3/500
4/4 [==============================] - 0s 4ms/step - loss: 1.8070 - accuracy: 0.1538
Epoch 4/500
4/4 [==============================] - 0s 4ms/step - loss: 1.8038 - accuracy: 0.2308
Epoch 5/500
4/4 [==============================] - 0s 3ms/step - loss: 1.8003 - accuracy: 0.2308
Epoch 6/500
4/4 [==============================] - 0s 4ms/step - loss: 1.7979 - accuracy: 0.2308
Epoch 7/500
4/4 [==============================] - 0s 4ms/step - loss: 1.7950 - accuracy: 0.2308
Epoch 8/500
4/4 [==============================] - 0s 4ms/step - loss: 1.7917 - accuracy: 0.2692
Epoch 9/500
4/4 [==============================] - 0s 4ms/step - loss: 1.7888 - accuracy: 0.2692
Epoch 10/500
4/4 [==============================] - 0s 4ms/step - loss: 1.7863 - accuracy: 0.2692
Epoch 11/500
4/4 [===========

In [ ]:
def bag_of_words(sent,words):
  bag=[0 for _ in range(len(words))]

  s_words= nltk.word_tokenize(sent)
  s_words= [stemmer.stem(wd.lower()) for wd in s_words]

  for s in s_words:
    for i,w in enumerate(words):
      if w==s:
        bag[i]=1
  
  return np.array(bag)

In [ ]:
def chat():
  print('Chatbot is available(type quit to stop)')

  while True:
    imp= input('You: ')
    if imp.lower()== 'quit':
      break
    
    bag=bag_of_words(imp,words)
    results= model.predict(np.reshape([bag], (1, 46)))[0]
    results_index = np.argmax(results)
    
    tag= labels[results_index]

    if results[results_index] >= 0.7:

       for tg in data['intents']:
         if tg['tag']==tag:
           response= tg['responses']
   
       print(random.choice(response))

    else:
      print("I didn't that, get please try again.")
  
chat()

Chatbot is available(type quit to stop)
You: hi
Hi there, how can I help?
You: goodbye
Talk to you later
You: quit
